In [2]:
from googleapiclient.discovery import build
from datetime import date
import pandas as pd
import boto3
import io
import time

In [3]:
key = pd.read_csv('/Users/ha/Project/AWS/hahoang-local_accessKeys.csv')
YOUTUBE_API_KEY = key['Youtube Key'][0]
AWS_KEY_ID = key['Access key ID'][0]
AWS_SECRET = key['Secret access key'][0]
AWS_REGION = 'ap-southeast-1'

In [27]:
api_service_name = "youtube"
api_version = "v3"
# create an API client
youtube = build(
    api_service_name, api_version, developerKey=YOUTUBE_API_KEY)

# make a request for the top videos
request = youtube.videos().list(
    part="snippet,contentDetails,statistics",
    chart="mostPopular",
    regionCode='PH',
    maxResults=50
)
response = request.execute()

In [25]:
response['items']

[{'kind': 'youtube#video',
  'etag': 'xDH0z-3HLUj9fVoGQ1ywHb_d2Ew',
  'id': 'r0rMch9h65c',
  'snippet': {'publishedAt': '2023-06-23T12:14:31Z',
   'channelId': 'UC0qTInmgyYnwWLeCUIde9Cw',
   'title': 'Dép chống trơn trượt có hiệu quả như lời đồn | Mẹ Hương Hương',
   'description': '',
   'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/r0rMch9h65c/default.jpg',
     'width': 120,
     'height': 90},
    'medium': {'url': 'https://i.ytimg.com/vi/r0rMch9h65c/mqdefault.jpg',
     'width': 320,
     'height': 180},
    'high': {'url': 'https://i.ytimg.com/vi/r0rMch9h65c/hqdefault.jpg',
     'width': 480,
     'height': 360},
    'standard': {'url': 'https://i.ytimg.com/vi/r0rMch9h65c/sddefault.jpg',
     'width': 640,
     'height': 480},
    'maxres': {'url': 'https://i.ytimg.com/vi/r0rMch9h65c/maxresdefault.jpg',
     'width': 1280,
     'height': 720}},
   'channelTitle': 'Mẹ Hương Hương',
   'categoryId': '22',
   'liveBroadcastContent': 'none',
   'localized': {'title': 'Dép

In [6]:
today = date.today()
df = pd.DataFrame(response['items'])

In [7]:
x = pd.DataFrame(df['snippet'].tolist())
y = pd.DataFrame(df['statistics'].tolist())

In [8]:
x.head(2)

,publishedAt,channelId,title,description,thumbnails,channelTitle,categoryId,liveBroadcastContent,localized,tags,defaultAudioLanguage,defaultLanguage
0,2023-06-23T12:14:31Z,UC0qTInmgyYnwWLeCUIde9Cw,Dép chống trơn trượt có hiệu quả như lời đồn |...,,{'default': {'url': 'https://i.ytimg.com/vi/r0...,Mẹ Hương Hương,22,none,{'title': 'Dép chống trơn trượt có hiệu quả nh...,NaN,NaN,NaN
1,2023-06-24T14:03:35Z,UCtUWP_zV5IDdiz6W97R2Zpg,Khi tôi làm mẹ kiểu: @rufinoaybar - Andrea A...,#andreaaybar #vlog\nKênh Youtube chính thức củ...,{'default': {'url': 'https://i.ytimg.com/vi/tm...,Andrea Aybar,22,none,{'title': 'Khi tôi làm mẹ kiểu: @rufinoaybar ...,"[andrea aybar, andrea aybar tiktok, andrea ayb...",vi,NaN


In [9]:
y.head(2)

,viewCount,likeCount,favoriteCount,commentCount
0,5179148,74983,0,164
1,3441268,88084,0,1520


In [10]:
df = pd.concat([df.drop('snippet', axis=1), pd.DataFrame(df['snippet'].tolist())], axis=1)
df = pd.concat([df.drop('statistics', axis=1), pd.DataFrame(df['statistics'].tolist())], axis=1)
df.columns

Index(['kind', 'etag', 'id', 'contentDetails', 'publishedAt', 'channelId',
       'title', 'description', 'thumbnails', 'channelTitle', 'categoryId',
       'liveBroadcastContent', 'localized', 'tags', 'defaultAudioLanguage',
       'defaultLanguage', 'viewCount', 'likeCount', 'favoriteCount',
       'commentCount'],
      dtype='object')

In [11]:
# replace the category id with the name
dict_category = {1 : 'Film & Animation', 2 : 'Autos & Vehicles', 10 : 'Music', 15 : 'Pets & Animals', 17 : 'Sports', 18 : 'Short Movies', 19 : 'Travel & Events',
                 20 : 'Gaming', 21 : 'Videoblogging', 22 : 'People & Blogs', 23 : 'Comedy', 24 : 'Entertainment', 25 : 'News & Politics', 26 : 'Howto & Style', 27 : 'Education',
                 28 : 'Science & Technology', 29 : 'Nonprofits & Activism', 30 : 'Movies', 31 : 'Anime/Animation', 32 : 'Action/Adventure', 33 : 'Classics', 34 : 'Comedy',
                 35 : 'Documentary', 36 : 'Drama', 37 : 'Family', 38 : 'Foreign', 39 : 'Horror', 40 : 'Sci-Fi/Fantasy', 41 : 'Thriller', 42 : 'Shorts', 43 : 'Shows', 44 : 'Trailers'}
df['categoryId'] = df['categoryId'].astype(int)
df['category'] = df['categoryId'].map(dict_category)
df.insert(0, "date_of_extraction", today)
df.insert(1, "country", 'VN')

In [12]:
df.head(2)

,date_of_extraction,country,kind,etag,id,contentDetails,publishedAt,channelId,title,description,...,liveBroadcastContent,localized,tags,defaultAudioLanguage,defaultLanguage,viewCount,likeCount,favoriteCount,commentCount,category
0,2023-06-30,VN,youtube#video,UQQFy1u8ISS0Xf6wiahFNmi1pOs,r0rMch9h65c,"{'duration': 'PT48S', 'dimension': '2d', 'defi...",2023-06-23T12:14:31Z,UC0qTInmgyYnwWLeCUIde9Cw,Dép chống trơn trượt có hiệu quả như lời đồn |...,,...,none,{'title': 'Dép chống trơn trượt có hiệu quả nh...,NaN,NaN,NaN,5179148,74983,0,164,People & Blogs
1,2023-06-30,VN,youtube#video,LmTXbIpVE3iwcpnQPIYHdQo4_t4,tmqTFcrjqaw,"{'duration': 'PT25S', 'dimension': '2d', 'defi...",2023-06-24T14:03:35Z,UCtUWP_zV5IDdiz6W97R2Zpg,Khi tôi làm mẹ kiểu: @rufinoaybar - Andrea A...,#andreaaybar #vlog\nKênh Youtube chính thức củ...,...,none,{'title': 'Khi tôi làm mẹ kiểu: @rufinoaybar ...,"[andrea aybar, andrea aybar tiktok, andrea ayb...",vi,NaN,3441268,88084,0,1520,People & Blogs


In [13]:
columns = ['date_of_extraction', 'country','id', 'title', 'description', 'channelId', 'channelTitle', 'category', 'viewCount', 'likeCount', 'favoriteCount', 'commentCount']
df = df[columns]
df = df.dropna()
int_col = df.columns[-4:]
for col in int_col:
    df[col] = df[col].astype(int)
str_col = df.columns[:-4]
for col in str_col:
    df[col] = df[col].astype(str)

df = df[['date_of_extraction',  'country','id', 'title',  'channelId','channelTitle', 'category', 'viewCount', 'likeCount', 'favoriteCount', 'commentCount']]
print(len(df.columns))
print(df.dtypes)

11
date_of_extraction    object
country               object
id                    object
title                 object
channelId             object
channelTitle          object
category              object
viewCount              int64
likeCount              int64
favoriteCount          int64
commentCount           int64
dtype: object


In [14]:
df.head(2)

,date_of_extraction,country,id,title,channelId,channelTitle,category,viewCount,likeCount,favoriteCount,commentCount
0,2023-06-30,VN,r0rMch9h65c,Dép chống trơn trượt có hiệu quả như lời đồn |...,UC0qTInmgyYnwWLeCUIde9Cw,Mẹ Hương Hương,People & Blogs,5179148,74983,0,164
1,2023-06-30,VN,tmqTFcrjqaw,Khi tôi làm mẹ kiểu: @rufinoaybar - Andrea A...,UCtUWP_zV5IDdiz6W97R2Zpg,Andrea Aybar,People & Blogs,3441268,88084,0,1520


In [29]:
s3 = boto3.client(service_name='s3',
                    region_name='ap-southeast-1',
                    aws_access_key_id = AWS_KEY_ID,
                    aws_secret_access_key = AWS_SECRET)

In [31]:
athena_client = boto3.client('athena',
                region_name='ap-southeast-1',
                aws_access_key_id = AWS_KEY_ID,
                aws_secret_access_key = AWS_SECRET)